In [14]:
import numpy as np
import matplotlib.pyplot as plt

# Exercises

These exercises should help you become more familiar with the Black-Scholes formula, which might seem rather intimidating at first. They also
give more simple examples of replication arguments.

## Exercise

We have defined a European call option to be a derivative contract that pays off $\max\{S_T-K,0\}$ at time $T$. A European call option with strike $K$ and maturity $T$ is often described as "the right (but not the obligation) to buy the stock at time $T$ at a pre-determined price $K$." Explain why having such a right is equivalent to receiving a payment of $\max\{S_T-K,0\}$. Note that it is a right but not an obligation, so if you lose money you would not take up this right. This is why it is called an *option*, you have the *option* of whether or not to *exercise* your right at time $T$. 

If $S_T>K$ then at maturity you should exercise your right to buy the stock for $K$ and immediately sell it giving you an instantaneous profit of $S_T-K$. If instead $S_T < K$ and you were to exercise your right, you would make an instantaneous loss of $S_T-K$, so you should not exercise your right.

Thus the profit you would make at time $T$ if you owned a European call option will be
$$
\begin{split}
\text{Profit}&=
\begin{cases}
S_T-K & S_T \geq K \\
0 & S_T < K
\end{cases} \\
&=
\begin{cases}
S_T-K & S_T - K \geq 0  \\
0 & S_T - < 0
\end{cases} \\
&= \max\{S_T-K,0\}
\end{split}
$$

## Exercise

Plot a graph of the payoff of a European call option with strike $K$ and maturity $T$ against $S_T$, the final stock price. Take $K=100$ to be concrete.

In [19]:
### BEGIN SOLUTION
ST = np.linspace(0,200,100)
K = 100
payoff = np.maximum(ST-K,0)
ax = plt.gca()
ax.plot(ST, payoff)
ax.set_xlabel('$S_T$')
ax.set_ylabel('Payoff');
ax.set_title('Call option payoff');
### END SOLUTION

Text(0.5, 1.0, 'Call option payoff')

## Exercise

Suppose $K_1 < K_2$. Explain why a European call option with strike $K_1$ and maturity $T$ is worth more than a European call option with strike $K_2$ and maturity $T$.

If $K_1 < K_2$ then $S_T-K_1 > S_T-K_2$ and hence $\max \{S_T-K_1,0\}\geq \max\{S_T-K_2,0\}$. Thus the call option with strike $K_1$ will always have at least as large a payoff of as the call option with strike $K_2$. We say that the option with strike $K_1$ *super-replicates* the option with strike $K_2$ because it always has a larger payoff.

## Exercise

Plot a graph of the payoff of a European put option against $S_T$

In [20]:
### BEGIN SOLUTION
ST = np.linspace(0,200,100)
K = 100
payoff = np.maximum(K-ST,0)
ax = plt.gca()
ax.plot(ST, payoff)
ax.set_xlabel('$S_T$')
ax.set_ylabel('Payoff');
ax.set_title('Put option payoff');
### END SOLUTION

## Exercise

Consider a portfolio consisting of:
    
* $+1$ units of a European call option with strike $K$ and maturity $T$
* $-1$ units of a European put option with strike $K$ and maturity $T$

Plot a graph of the portfolio payoff against $S_T$. Deduce that such a portfolio can be replicated by purchasing $1$ unit of stock and putting
$e^{-rT} K$ into a risk-free bank account at time $0$.

Deduce the following:
$$\text{Price of call at time 0}-\text{Price of put at time 0}=S_0-e^{-rT}K$$ 

In [23]:
### BEGIN SOLUTION
ST = np.linspace(0,200,100)
K = 100
payoff = np.maximum(ST-K,0)-np.maximum(K-ST,0)
ax = plt.gca()
ax.plot(ST, payoff)
ax.set_xlabel('$S_T$')
ax.set_ylabel('Payoff');
ax.set_title('Portfolio payoff');
### END SOLUTION

We see that the portfolio payoff is equal to $S-K$. A payoff of $S$ can be replicated by buying the stock. A payoff of $K$ can be replicated by putting $e^{-rT}K$ into the risk-free bank account at time $0$. 

## Exercise

Plot a graphs of the Black-Scholes price of an option against the stock price $S_t$ for a European call option
with strike $K=100$, maturity $T=1$ in a model where
$r=0.02$, $\sigma=0.2$ and $S_0=110$. You should plot graphs for values $t \in \{0, \frac{1}{10}, \frac{2}{10}, \ldots, \frac{9}{10} \}$.

In [32]:
St = np.linspace(80,120,100)
import fmlib
ax = plt.gca()
for i in range(0,10):
    t = i/10
    V = fmlib.black_scholes_call_price(St,t,100,1, 0.02, 0.2)
    ax.plot(St,V, label='t={}'.format(t))
ax.legend();

## Exercise

Let $d_i(S,t,K,T,r,\sigma)$ be as in the Black-Scholes formula. Suppose that $S>K$. Show that
$$\lim_{t \to T} d_1(S,t,K,T,r,\sigma)=\infty$$
and
$$\lim_{t \to T} d_2(S,t,K,T,r,\sigma)=\infty.$$

Deduce that
$$ \lim_{t \to T} V(S,t,K,T,r,\sigma)=S-K $$
if $S>K$. What is the value of
$$ \lim_{t \to T} V(S,t,K,T,r,\sigma)=S-K $$
if $ S < K $?

If $S>K$, $\log(S/K)>0$. We have
$$d_1 = \frac{1}{\sigma \sqrt{T-t}} \log(S/K) + (r+\frac{1}{2}\sigma^2)\frac{1}{\sigma} \sqrt{T-t}$$
The first term tends to $\infty$ as $t \to T$ since $\log(S/K)$ is positive and $\sqrt{T-t}\to 0$. The second term tends to 0. Hence $d_1 \to \infty$. Similarly $d_2 \to \infty$.

We deduce that $N(d_1) \to 1$ and $N(d_2) \to 1$ as $t \to T$ (since $N$ is a distribution function). Hence $V=S N(d_1) - e^{-r(T-t)} N(d_2) \to S - K$.

If $S < K$, $\log(S/K)<0$, so $d_1$ and $d_2$ both tend to $-\infty$. Hence $N(d_1)$ and $N(d_2)$ both tend to $0$, so $V \to 0$.

In general case $\lim_{t \to T}V = \max\{S-K,0\}$. 
